In [26]:
# Imports
from biocrnpyler import *
from genelet import *
import pylab as plt
import numpy as np
from bokeh.layouts import row

import warnings
import bokeh.io
import bokeh.plotting

## CRNs
### NAND Gate CRN

In [27]:
# Creating CRN for NAND gate

S1_off = Species("Sw1")
S2_off = Species("Sw2")
S3_off = Species("Sw3")

S1 = Genelet(S1_off, transcript = "I31", activator = "A1", inhibitor = "I1" )
S2 = Genelet(S2_off, transcript = "I32", activator = "A2", inhibitor = "I2" )
S3 = Genelet(S3_off, transcript = "I4", activator = "A31", inhibitor = "I31", activator2 = "A32", inhibitor2 = "I32" )

M_NAND = Mixture(name = "Switch_test", components = [S1,S2,S3], parameter_file = "default_parameters.txt")


repr(M_NAND)
CRN_NAND = M_NAND.compile_crn()
#print(CRN_NAND.pretty_print())

### AND Gate CRN

In [28]:
S4_off = Species("Sw4")

S4 = Genelet(S4_off, transcript = "P", activator = "A4", inhibitor = "I4" )

M_AND = Mixture(name = "AND_test", components = [S1,S2,S3,S4], parameter_file = "default_parameters.txt")

repr(M_AND)
CRN_AND = M_AND.compile_crn()
#print(CRN_AND.pretty_print())

### NOR Gate CRN

In [29]:
# Creating CRN for NOR gate

S1_off = Species("Sw1")
S2_off = Species("Sw2")
S3_off = Species("Sw3")

S1 = Genelet(S1_off, transcript = "I3", activator = "A1", inhibitor = "I1" )
S2 = Genelet(S2_off, transcript = "I3", activator = "A2", inhibitor = "I2" )
S3 = Genelet(S3_off, transcript = "I4", activator = "A3", inhibitor = "I3" )

M_NOR = Mixture(name = "Switch_test", components = [S1,S2,S3], parameter_file = "default_parameters.txt")

repr(M_NOR)
CRN_NOR = M_NOR.compile_crn()
#print(CRN_NOR.pretty_print())

### OR Gate CRN

In [30]:
S4_off = Species("Sw4")

S4 = Genelet(S4_off, transcript = "P", activator = "A4", inhibitor = "I4" )

M_OR = Mixture(name = "AND_test", components = [S1,S2,S3,S4], parameter_file = "default_parameters.txt")

repr(M_OR)
CRN_OR = M_OR.compile_crn()
#print(CRN_OR.pretty_print())

## Simulations
### NAND Gate Simulation

In [37]:
# Bioscrape simulation of above CRN

io = {"Sw1_OFF": 2000, "dna_A1": 2000, "rna_I1": 0, "Sw2_OFF": 2000, "dna_A2": 2000, "rna_I2": 0, "Sw3_OFF":2000, "dna_A31": 2000, "dna_A32": 2000, "protein_RNAseH":20,
      "protein_RNAP":150}
timepoints = np.linspace(0, 5000, 1000)
R = CRN_NAND.simulate_with_bioscrape(timepoints, initial_condition_dict = io)


bokeh.io.output_notebook()
p = bokeh.plotting.figure(plot_width=300, plot_height=300)
p.circle(timepoints, R["Sw1_OFF"], legend_label = "OFF switch 1", color = "orange")
p.circle(timepoints, R["complex_Sw1_ON"], legend_label = "ON switch 1", color = "red")
p.legend.click_policy="hide"


s = bokeh.plotting.figure(plot_width=300, plot_height=300)
s.circle(timepoints, R["Sw2_OFF"], legend_label = "OFF switch 2", color = "orange")
s.circle(timepoints, R["complex_Sw2_ON"], legend_label = "ON switch 2" , color = "red")
s.legend.click_policy="hide"


# Total amount of activated and deactivated forms of Switch 3
q = bokeh.plotting.figure(plot_width=300, plot_height=300)
q.circle(timepoints, R["Sw3_OFF"], legend_label = "OFF switch 3", color = "orange")
q.circle(timepoints, R["complex_Sw3_ON_1"]+R["complex_Sw3_ON_2"], legend_label = "ON switch 3" , color = "red")
q.legend.click_policy="hide"



r = bokeh.plotting.figure(plot_width=300, plot_height=300)
r.circle(timepoints, R["rna_I4"], legend_label = "Inhibitor 4", color = "green")
r.circle(timepoints, R["rna_I31"], legend_label = "Inhibitor 31" , color = "red")
r.circle(timepoints, R["rna_I32"], legend_label = "Inhibitor 32" , color = "blue")
r.circle(timepoints, R["complex_Sw3_AI"], legend_label = "Complex 31", color = "purple")
r.legend.click_policy="hide"

bokeh.io.show(row(p, s, q,r))
warnings.filterwarnings("ignore")

odeint failed with mxstep=500...

Loading BokehJS ...

In [32]:
q = bokeh.plotting.figure(plot_width=300, plot_height=300)
q.circle(timepoints, R["Sw3_OFF"], legend_label = "OFF switch 3", color = "orange")
q.circle(timepoints, R["complex_Sw3_ON_1"], legend_label = "ON switch 3 - 1" , color = "red")
q.circle(timepoints, R["complex_Sw3_ON_2"], legend_label = "ON switch 3 - 2" , color = "blue")
q.legend.click_policy="hide"
bokeh.io.show(q)

### NOR Gate Simulation

In [33]:
# Bioscrape simulation of NOR gate CRN

io = {"Sw1_OFF": 2000, "dna_A1": 2000, "rna_I1": 0, "Sw2_OFF": 2000, "dna_A2": 2000, "rna_I2": 0, "Sw3_OFF":2000, "dna_A3": 2000,
      "rna_I3": 0, "protein_RNAseH":10, "protein_RNAP":500}

timepoints = np.linspace(0, 500, 1000)
R = CRN_NOR.simulate_with_bioscrape(timepoints, initial_condition_dict = io)

bokeh.io.output_notebook()
p = bokeh.plotting.figure(plot_width=300, plot_height=300)
p.circle(timepoints, R["Sw1_OFF"], legend_label = "OFF switch 1", color = "orange")
p.circle(timepoints, R["complex_Sw1_ON"], legend_label = "ON switch 1", color = "red")

s = bokeh.plotting.figure(plot_width=300, plot_height=300)
s.circle(timepoints, R["Sw2_OFF"], legend_label = "OFF switch 1", color = "orange")
s.circle(timepoints, R["complex_Sw2_ON"], legend_label = "ON switch 2" , color = "red")

q = bokeh.plotting.figure(plot_width=300, plot_height=300)
q.circle(timepoints, R["Sw3_OFF"], legend_label = "OFF switch 3", color = "orange")
q.circle(timepoints, R["complex_Sw3_ON"], legend_label = "ON switch 3" , color = "red")

bokeh.io.show(row(p, s, q))
warnings.filterwarnings("ignore")

Loading BokehJS ...

In [34]:
# NOR gate activators and inhibitors of importance

bokeh.io.output_notebook()

p = bokeh.plotting.figure(plot_width=300, plot_height=300)
p.circle(timepoints, R["dna_A3"], legend_label = "Activator 3", color = "orange")
p.circle(timepoints, R["rna_I3"], legend_label = "Inhibitor 3", color = "red")
p.circle(timepoints, R["complex_Sw3_AI"], legend_label = "Complex 3", color = "blue")
p.legend.click_policy="hide"

s = bokeh.plotting.figure(plot_width=300, plot_height=300)
s.circle(timepoints, R["rna_I4"], legend_label = "Inhibitor 4", color = "red")
s.legend.click_policy="hide"

bokeh.io.show(row(p, s))
warnings.filterwarnings("ignore")

Loading BokehJS ...

### AND Gate Simulation

In [35]:
# Bioscrape simulation of above CRN

io = {"Sw1_OFF": 2000, "dna_A1": 2000, "rna_I1": 0, "Sw2_OFF": 2000, "dna_A2": 2000, "rna_I2": 0, "Sw3_OFF":2000, "dna_A31": 2000, "dna_A32": 2000, "protein_RNAseH":20,
      "protein_RNAP":150}
timepoints = np.linspace(0, 5000, 1000)
R = CRN_AND.simulate_with_bioscrape(timepoints, initial_condition_dict = io)


bokeh.io.output_notebook()
p = bokeh.plotting.figure(plot_width=300, plot_height=300)
p.circle(timepoints, R["Sw1_OFF"], legend_label = "OFF switch 1", color = "orange")
p.circle(timepoints, R["complex_Sw1_ON"], legend_label = "ON switch 1", color = "red")
p.legend.click_policy="hide"

s = bokeh.plotting.figure(plot_width=300, plot_height=300)
s.circle(timepoints, R["Sw2_OFF"], legend_label = "OFF switch 2", color = "orange")
s.circle(timepoints, R["complex_Sw2_ON"], legend_label = "ON switch 2" , color = "red")
s.legend.click_policy="hide"

# Total amount of activated and deactivated forms of Switch 3
q = bokeh.plotting.figure(plot_width=300, plot_height=300)
q.circle(timepoints, R["Sw3_OFF"], legend_label = "OFF switch 3", color = "orange")
q.circle(timepoints, R["complex_Sw3_ON_1"]+R["complex_Sw3_ON_2"], legend_label = "ON switch 3" , color = "red")
q.legend.click_policy="hide"

r = bokeh.plotting.figure(plot_width=300, plot_height=300)
r.circle(timepoints, R["rna_I4"], legend_label = "Inhibitor 4", color = "green")
r.circle(timepoints, R["rna_I31"], legend_label = "Inhibitor 31" , color = "red")
r.circle(timepoints, R["rna_I32"], legend_label = "Inhibitor 32" , color = "blue")
r.circle(timepoints, R["complex_Sw3_AI"], legend_label = "Complex 31", color = "purple")
r.legend.click_policy="hide"
bokeh.io.show(row(p, s, q,r))
warnings.filterwarnings("ignore")

odeint failed with mxstep=500...

Loading BokehJS ...

In [36]:
q = bokeh.plotting.figure(plot_width=300, plot_height=300)
q.circle(timepoints, R["Sw3_OFF"], legend_label = "OFF switch 3", color = "orange")
q.circle(timepoints, R["complex_Sw3_ON_1"], legend_label = "ON switch 3 - 1" , color = "red")
q.circle(timepoints, R["complex_Sw3_ON_2"], legend_label = "ON switch 3 - 2" , color = "blue")
q.legend.click_policy="hide"

r = bokeh.plotting.figure(plot_width=300, plot_height=300)
r.circle(timepoints, R["rna_I4"], legend_label = "Inhibitor 4", color = "green")
r.circle(timepoints, R["rna_I31"], legend_label = "Inhibitor 31" , color = "red")
r.circle(timepoints, R["rna_I32"], legend_label = "Inhibitor 32" , color = "blue")
r.circle(timepoints, R["complex_Sw3_AI"], legend_label = "Complex 31", color = "purple")
r.legend.click_policy="hide"
bokeh.io.show(row(q, r))